# Analyzing neuropil signal and frequency composition
* This is the batch version of the neuropil analysis notebook. It is meant to be run on more experiments.

In [1]:
import sys, os
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import scipy.signal as signal
from scipy.interpolate import interp1d

sys.path.append("/home/gergely/code/sleep/src/")

from suite2p_class import Suite2p
from imaging_data_class import ImagingData
from eeg_class import eegData
from behavior_class import behaviorData
import dfof

In [ ]:
# setting up dfof calulations
s2p_calc_dfof = dfof.Suite2pDFOF(window=1800)
jia_calc_dfof = dfof.JiaDFOF(slow_trend_window=1800)
# s2p_mouse = Suite2p()

In [2]:
mice = ["5HT2afl05b_1", "5HT2afl05b_2", "sert52b_1", "sert52b_5", "sert54a_5"]
sima_folders = [
    "/data2/gergely/invivo_DATA/sleep/5HT2afl05b_1/7_7_SD/TSeries-07072021-0833_GC1-001/TSeries-07072021-0833_GC1-001_Cycle00001_Element00001.sima",
    "/data2/gergely/invivo_DATA/sleep/5HT2afl05b_1/7_7_SD/TSeries-07072021-0833_GC2-003/TSeries-07072021-0833_GC2-003_Cycle00001_Element00001.sima",
    "/data2/gergely/invivo_DATA/sleep/5HT2afl05b_1/7_7_SD/TSeries-07072021-0833_MC1-002/TSeries-07072021-0833_MC1-002_Cycle00001_Element00001.sima",
    "/data2/gergely/invivo_DATA/sleep/5HT2afl05b_1/7_7_SD/TSeries-07072021-0833_MC2-004/TSeries-07072021-0833_MC2-004_Cycle00001_Element00001.sima",
    "/data2/gergely/invivo_DATA/sleep/5HT2afl05b_2/7_6_SD/TSeries-07062021-0844_GC-001/TSeries-07062021-0844_GC-001_Cycle00001_Element00001.sima",
    "/data2/gergely/invivo_DATA/sleep/5HT2afl05b_2/7_6_SD/TSeries-07062021-0844_MC-002/TSeries-07062021-0844_MC-002_Cycle00001_Element00001.sima",
    "/data2/gergely/invivo_DATA/sleep/5HT2afl05b_2/7_6_SD/TSeries-07062021-0844_MC2-004/TSeries-07062021-0844_MC2-004_Cycle00001_Element00001.sima",
    "/data2/gergely/invivo_DATA/sleep/sert52b_5/4_3_SD/TSeries-04032023-1323-001/TSeries-04032023-1323-001.sima",
    "/data2/gergely/invivo_DATA/sleep/sert54a_5/4_4/TSeries-04042023-1023_sess1-001/TSeries-04042023-1023_sess1-001.sima",
    "/data2/gergely/invivo_DATA/sleep/sert52b_1/4_3_SD/TSeries-04032023-1323_SD_sess1-001/TSeries-04032023-1323_SD_sess1-001.sima",
]

In [3]:
def freq_calc(data: pd.Series, fs: int = 10):
    """data: npil_eeg data"""
    frequencies, psd = signal.welch(data, fs=fs, detrend="linear")
    return frequencies, psd


def plot_nrem_awake_immob(data, df, data_name, sima_folder):
    fig, ax = plt.subplots(nrows=3, figsize=(20, 5), sharey=True)
    # concatentate the True values:
    ax[0].plot(data.values, color="k", label="all cells")
    ax[0].legend([f"all {data_name}"], loc="upper right", labelcolor="k")
    ax[1].plot(data[df["awake_immobile"]].values, label="awake")
    ax[1].legend(loc="upper right", labelcolor="k")
    ax[2].plot(data[df["NREM"]].values, color="orange")
    ax[2].legend(["NREM"], loc="upper right", labelcolor="k")
    fig.savefig(
        join(sima_folder, f"NREM Awake_immob Mean {data_name} traces.png"),
        bbox_inches="tight",
        dpi=300,
    )


def plot_semilogy1(data, sima_folder):
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), sharey=True)
    fig.suptitle("Power Spectral Density: Awake vs NREM")

    # Plot the first semilogy plot in the first subplot
    axs[0].semilogy(
        data["awake_cell"]["freq"], data["awake_cell"]["psd"], label="awake"
    )
    axs[0].semilogy(data["nrem_cell"]["freq"], data["nrem_cell"]["psd"], label="nrem")
    axs[0].legend(loc="lower left")

    # Plot the second semilogy plot in the first subplot
    axs[1].semilogy(
        data["awake_npil"]["freq"], data["awake_npil"]["psd"], label="awake"
    )
    axs[1].semilogy(data["nrem_npil"]["freq"], data["nrem_npil"]["psd"], label="nrem")
    axs[1].legend(loc="lower left")

    # Set the title and axis labels for the first subplot
    axs[0].set_title("Cell signal")
    axs[0].set_xlabel("Frequency (Hz)")
    axs[0].set_ylabel("Power/Frequency (dB/Hz)")

    # Set the title and axis labels for the second subplot
    axs[1].set_title("Npil signal")
    axs[1].set_xlabel("Frequency (Hz)")
    fig.savefig(
        join(sima_folder, "PSD plot cell vs npil.png"), bbox_inches="tight", dpi=300
    )


def plot_semilogy2(data, sima_folder):
    # Create a figure with 2 subplots in 1 row and 2 columns
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), sharey=True)
    fig.suptitle("Power Spectral Density: Cell vs Npil")

    # Plot the first semilogy plot in the first subplot
    axs[0].semilogy(
        results["awake_cell"]["freq"], results["awake_cell"]["psd"], label="cell"
    )
    axs[0].semilogy(
        results["awake_npil"]["freq"], results["awake_npil"]["psd"], label="npil"
    )
    axs[0].legend()

    # Plot the second semilogy plot in the first subplot
    axs[1].semilogy(
        results["nrem_cell"]["freq"], results["nrem_cell"]["psd"], label="cell"
    )
    axs[1].semilogy(
        results["nrem_npil"]["freq"], results["nrem_npil"]["psd"], label="npil"
    )
    axs[1].legend()

    # Set the title and axis labels for the first subplot
    axs[0].set_title("Awake signal")
    axs[0].set_xlabel("Frequency (Hz)")
    axs[0].set_ylabel("Power/Frequency (dB/Hz)")

    # Set the title and axis labels for the second subplot
    axs[1].set_title("NREM signal")
    axs[1].set_xlabel("Frequency (Hz)")
    fig.savefig(
        join(sima_folder, "PSD plot awake vs NREM.png"), bbox_inches="tight", dpi=300
    )

In [ ]:
for mouse in mice:
    m = Mouse(mouse)
    s2p_folders = m.find_suite2p_folders()
    for s2p_folder in s2p_folders:
        if os.path.dirname(s2p_folder) in sima_folders:
            s2p_data = Suite2p(s2p_folder=s2p_folder)
            print(s2p_folder)
            sima_folder = os.path.dirname(s2p_folder)

            # loading the eeg data
            file_name = join(sima_folder, "eeg", "velo_eeg.csv")
            eeg_velo = eegData.load_processed_velocity_eeg(file_name=file_name)
            eeg_velo["mobile_immobile"] = behaviorData.define_immobility(
                eeg_velo["filtered velo"])
            df = eegData.brain_state_filter(
                velo_eeg_df=eeg_velo,
                states=["awake_mobile", "awake_immobile", "NREM", "REM", "other"],
            )

            # loading the suite2p data
            # The class initializtion must be here and need to be reinitialized for each mouse
            s2p_calc_dfof = dfof.Suite2pDFOF(window=1800)
            jia_calc_dfof = dfof.JiaDFOF(slow_trend_window=1800)
            s2p_data = Suite2p(s2p_folder=s2p_folder)
            cells = s2p_data.cells()
            npil = s2p_data.npil()
            cell_dfof = s2p_calc_dfof.calculate(signal=cells, npil=npil)
            npil_dfof = jia_calc_dfof.calculate(signal=npil)
            mean_cellF = cell_dfof.mean(axis=0)
            mean_npilF = npil_dfof.mean(axis=0)

            saving the plots in the sima folders
            plot_nrem_awake_immob(
                data=mean_cellF, df=df, data_name="cells", sima_folder=sima_folder
            )
            plot_nrem_awake_immob(
                data=mean_npilF, df=df, data_name="npil", sima_folder=sima_folder
            )

            # calculating the PSD and frequencie
            input_data = {
                "nrem_cell": pd.Series(mean_cellF[df["NREM"]].values),
                "nrem_npil": pd.Series(mean_npilF[df["NREM"]].values),
                "awake_npil": pd.Series(mean_npilF[df["awake_immobile"]].values),
                "awake_cell": pd.Series(mean_cellF[df["awake_immobile"]].values),
            }

            results = {}

            for key, value in input_data.items():
                freq, psd = freq_calc(value)
                results[key] = {"freq": freq, "psd": psd}

            plot_semilogy1(data=results, sima_folder=sima_folder)
            plot_semilogy2(data=results, sima_folder=sima_folder)